In [ ]:
import pandas as pd   
import numpy as np    
import matplotlib.pyplot as plt 
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.metrics import auc, roc_curve, roc_auc_score, precision_recall_curve
from sklearn.metrics import fbeta_score, cohen_kappa_score
SEED = 42

In [ ]:

df = pd.read_csv('modify.csv')
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,125,33.6,0.627,50,1
1,1,85,66,29,125,26.6,0.351,31,0
2,8,183,64,29,125,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,4,137,40,35,168,43.1,2.288,33,1


In [ ]:

X = df.drop('Outcome', axis=1)   # axis=0 for row, axis=1 for column
y = df['Outcome']

# split data to 80:20 ratio for train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=SEED, stratify=y)
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)


X_train (614, 8)
y_train (614,)
X_test (154, 8)
y_test (154,)


In [ ]:
def model_report(model, X_train, X_test, y_train, y_test, model_name):
  
  model.fit(X_train,y_train)
  y_pred=model.predict(X_train)
  precision    = precision_score(y_train, y_pred)
  recall       = recall_score(y_train, y_pred)
  f1score      = f1_score(y_train, y_pred) 
  rocauc       = roc_auc_score(y_train, y_pred)
  logloss      = log_loss(y_train, y_pred)
  accuracy     = accuracy_score(y_train, y_pred)
  df_model = pd.DataFrame({  'model'        : [model_name],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             #'timetaken'    : [time1]
                           })  
  return df_model
   # timetaken for comparison later
def model_report1(model, X_train, X_test, y_train, y_test, model_name):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [model_name],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             #'timetaken'    : [0]   
                                })  
  
  return df_model1
logit = LogisticRegression()
knn = KNeighborsClassifier()
decisiontree = DecisionTreeClassifier()
extra = ExtraTreesClassifier()
randomforest = RandomForestClassifier()
svc = SVC()
linearsvc = LinearSVC()
gnb = GaussianNB()

# to concat all models
print("training pergormance metrics")
df_baseline = pd.concat([model_report(logit, X_train, X_test, y_train, y_test, 'Logistic Regresion'),
                       model_report(knn, X_train, X_test, y_train, y_test, 'KNN'),
                       model_report(gnb, X_train, X_test, y_train, y_test, 'GaussianNB'),
                       model_report(decisiontree, X_train, X_test, y_train, y_test, 'DecisionTree'),
                       model_report(extra, X_train, X_test, y_train, y_test, 'ExtraTrees'),
                       model_report(randomforest, X_train, X_test, y_train, y_test, 'RandomForest'),
                       model_report(linearsvc, X_train, X_test, y_train, y_test, 'LinearSVC'),
                       model_report(svc, X_train, X_test, y_train, y_test, 'SVC'),                 
                       ], axis=0).reset_index()
df_baseline = df_baseline.drop('index', axis=1)
df_baseline
print("Testing performce")


df_baseline1 = pd.concat([model_report1(logit, X_train, X_test, y_train, y_test, 'Logistic Regresion'),
                       model_report1(gnb, X_train, X_test, y_train, y_test, 'GaussianNB'),
                       model_report1(knn, X_train, X_test, y_train, y_test, 'KNN'),
                       model_report1(decisiontree, X_train, X_test, y_train, y_test, 'DecisionTree'),
                       model_report1(extra, X_train, X_test, y_train, y_test, 'ExtraTrees'),
                       model_report1(randomforest, X_train, X_test, y_train, y_test, 'RandomForest'),
                       model_report1(linearsvc, X_train, X_test, y_train, y_test, 'LinearSVC'),
                       model_report1(svc, X_train, X_test, y_train, y_test, 'SVC'),                 
                       ], axis=0).reset_index()
df_baseline1 = df_baseline1.drop('index', axis=1)
df_baseline1

training pergormance metrics
Testing performce


,model,accuracy,precision,recall,f1score,rocauc,logloss
0,Logistic Regresion,0.740260,0.659091,0.537037,0.591837,0.693519,8.971189
1,GaussianNB,0.701299,0.566667,0.629630,0.596491,0.684815,10.316912
2,KNN,0.688312,0.560000,0.518519,0.538462,0.649259,10.765447
3,DecisionTree,0.675325,0.543478,0.462963,0.500000,0.626481,11.213997
4,ExtraTrees,0.714286,0.600000,0.555556,0.576923,0.677778,9.868326
5,RandomForest,0.753247,0.666667,0.592593,0.627451,0.716296,8.522638
6,LinearSVC,0.649351,0.500000,0.074074,0.129032,0.517037,12.111020
7,SVC,0.720779,0.648649,0.444444,0.527473,0.657222,9.644012


In [ ]:
df_baseline #traning

,model,accuracy,precision,recall,f1score,rocauc,logloss
0,Logistic Regresion,0.799674,0.766082,0.612150,0.680519,0.756075,6.919058e+00
1,KNN,0.806189,0.751323,0.663551,0.704715,0.773026,6.694059e+00
2,GaussianNB,0.760586,0.675393,0.602804,0.637037,0.723902,8.269136e+00
3,DecisionTree,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
4,ExtraTrees,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
5,RandomForest,1.000000,1.000000,1.000000,1.000000,1.000000,9.992007e-16
6,LinearSVC,0.666124,0.714286,0.070093,0.127660,0.527547,1.153168e+01
7,SVC,0.778502,0.782609,0.504673,0.613636,0.714836,7.650322e+00


HYPERPARAMETER TUNING BY RANDOMIZED SEARCH


In [ ]:
! pip install sklearn_nature_inspired_algorithms==0.4.3
! pip install NiaPy
! pip install algorithms

In [ ]:
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(random_state=42)
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
%%time
time1 = time.time()
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

print("randomized model")

best_random = rf_random.best_estimator_
#df_random = pd.concat([evaluate(best_random,  X_test,  y_test, 'RandomForest')], axis=0).reset_index()
#df_random = df_random.drop('index', axis=1)

model11=evaluate(best_random,  X_test,  y_test, 'RandomForest')
model11.timetaken[0] = time.time() - time1
print(model11)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   56.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.3min finished


randomized model
          model  accuracy  precision  ...    rocauc   logloss  timetaken
0  RandomForest  0.753247   0.673913  ...  0.712037  8.522633  439.46194

[1 rows x 8 columns]
CPU times: user 5.11 s, sys: 315 ms, total: 5.43 s
Wall time: 7min 19s


In [ ]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 800}

In [ ]:
%%time
time2=time.time()
print("base model")
base_model = RandomForestClassifier()
base_model.fit(X_train, y_train)
#base_accuracy = evaluate(base_model, X_test, y_test,modelname)
model1 = evaluate(base_model,  X_test,  y_test, 'RandomForest')
model1.timetaken[0] = time.time() - time2
print(model1)


base model
          model  accuracy  precision  ...    rocauc   logloss  timetaken
0  RandomForest   0.74026   0.652174  ...  0.697778  8.971194   0.223264

[1 rows x 8 columns]
CPU times: user 225 ms, sys: 11.1 ms, total: 236 ms
Wall time: 234 ms


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(random_state=42)
print(et.get_params())

{'bootstrap': False, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid_et = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid_et)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
%%time
time1 = time.time()
etc = ExtraTreesClassifier(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
ETC = RandomizedSearchCV(estimator = etc, param_distributions = param_grid_et, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
ETC.fit(X_train, y_train)

best_ETC = ETC.best_estimator_
#df_random = pd.concat([evaluate(best_random,  X_test,  y_test, 'RandomForest')], axis=0).reset_index()
#df_random = df_random.drop('index', axis=1)

model12=evaluate(best_ETC,  X_test,  y_test, 'Extra Tree')
model12.timetaken[0] = time.time() - time1
print(model12)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   46.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.2min finished


        model  accuracy  precision  ...    rocauc   logloss   timetaken
0  Extra Tree  0.727273       0.62  ...  0.692037  9.419765  377.264013

[1 rows x 8 columns]
CPU times: user 5.91 s, sys: 336 ms, total: 6.24 s
Wall time: 6min 17s


In [ ]:
%%time
from sklearn.ensemble import ExtraTreesClassifier

time2=time.time()
print("base model")
base_model = ExtraTreesClassifier()
base_model.fit(X_train, y_train)
#base_accuracy = evaluate(base_model, X_test, y_test,modelname)
model2 = evaluate(base_model,  X_test,  y_test, 'Extra Tree')
model2.timetaken[0] = time.time() - time2
print(model2)


base model
        model  accuracy  precision  ...    rocauc   logloss  timetaken
0  Extra Tree  0.720779   0.612245  ...  0.682778  9.644043   0.176967

[1 rows x 8 columns]
CPU times: user 184 ms, sys: 2.92 ms, total: 187 ms
Wall time: 189 ms


In [ ]:
ETC.best_params_

{'bootstrap': False,
 'max_depth': 70,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 1600}

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(random_state=42)
print(lr.get_params())

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [ ]:
max_iter = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 100)]
C= [float(x) for x in np.linspace(start = 1, stop = 10, num = 10)]
solver=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
class_weight = [int(x) for x in np.linspace(1, 10, num = 10)]
class_weight.append(None)
n_jobs=[int(x) for x in np.linspace(1, 10, num = 10)]
n_jobs.append(None)
intercept_scaling = [1, 2, 4]
verbose=[0,1,2,3]
dual = [True, False]
multi_class=['auto', 'ovr', 'multinomial']
tol= [0.00001,0.0001,0.001,0.01,0.1,0,1]
# Create the random grid
param_grid_lr = {'max_iter': max_iter,
               'class_weight': class_weight,
               'solver': solver,
               'intercept_scaling': intercept_scaling,
               'dual': dual,
               'n_jobs' : n_jobs,
               'verbose' : verbose,
               'C' : C,
               'tol' : tol
               #'multi_class' : multi_class
              }
print(param_grid_lr)

{'max_iter': [200, 208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 305, 313, 321, 329, 337, 345, 353, 361, 369, 377, 385, 393, 402, 410, 418, 426, 434, 442, 450, 458, 466, 474, 482, 490, 498, 507, 515, 523, 531, 539, 547, 555, 563, 571, 579, 587, 595, 604, 612, 620, 628, 636, 644, 652, 660, 668, 676, 684, 692, 701, 709, 717, 725, 733, 741, 749, 757, 765, 773, 781, 789, 797, 806, 814, 822, 830, 838, 846, 854, 862, 870, 878, 886, 894, 903, 911, 919, 927, 935, 943, 951, 959, 967, 975, 983, 991, 1000], 'class_weight': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, None], 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'intercept_scaling': [1, 2, 4], 'dual': [True, False], 'n_jobs': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, None], 'verbose': [0, 1, 2, 3], 'C': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0], 'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1, 0, 1]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
%%time
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
time1 = time.time()
Lr = LogisticRegression(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
LR = RandomizedSearchCV(estimator = Lr, param_distributions = param_grid_lr, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
LR.fit(X_train, y_train)

best_LR = LR.best_estimator_
#df_random = pd.concat([evaluate(best_random,  X_test,  y_test, 'RandomForest')], axis=0).reset_index()
#df_random = df_random.drop('index', axis=1)

model13=evaluate(best_LR,  X_test,  y_test, 'Logistic Regression')
model13.timetaken[0] = time.time() - time1
print(model13)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    7.2s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   1 out of   1 | elapsed:    0.1s finished


                 model  accuracy  precision  ...    rocauc    logloss  timetaken
0  Logistic Regression  0.701299   0.583333  ...  0.659259  10.316881   7.526623

[1 rows x 8 columns]
CPU times: user 627 ms, sys: 88.4 ms, total: 715 ms
Wall time: 7.61 s


In [ ]:
LR.best_params_

{'C': 6.0,
 'class_weight': 7,
 'dual': False,
 'intercept_scaling': 1,
 'max_iter': 490,
 'n_jobs': 7,
 'solver': 'newton-cg',
 'tol': 1e-05,
 'verbose': 2}

In [ ]:
%%time
time2=time.time()
print("base model")
base_model = LogisticRegression()
base_model.fit(X_train, y_train)
#base_accuracy = evaluate(base_model, X_test, y_test,modelname)
model3 = evaluate(base_model,  X_test,  y_test, 'Logistic Regression')
model3.timetaken[0] = time.time() - time2
print(model3)


base model
                 model  accuracy  precision  ...    rocauc   logloss  timetaken
0  Logistic Regression   0.74026   0.659091  ...  0.693519  8.971189   0.187247

[1 rows x 8 columns]
CPU times: user 83.3 ms, sys: 1.99 ms, total: 85.3 ms
Wall time: 255 ms


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
kn=KNeighborsClassifier(n_neighbors=42)
print(kn.get_params())

{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 42, 'p': 2, 'weights': 'uniform'}


In [ ]:
leaf_size = [int(x) for x in np.linspace(start = 100, stop = 200, num = 20)]
#n_neighbors=[int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
#n_neighbors.append(None)
# Number of features to consider at every split
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
# Maximum number of levels in tree
p = [int(x) for x in np.linspace(1, 20, num = 20)]
param_grid_kn = {'leaf_size': leaf_size,
               'p': p,
               'algorithm': algorithm,
               #'n_neighbors' : n_neighbors,
               }
print(param_grid_kn)

{'leaf_size': [100, 105, 110, 115, 121, 126, 131, 136, 142, 147, 152, 157, 163, 168, 173, 178, 184, 189, 194, 200], 'p': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
%%time
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
time1 = time.time()
knn = KNeighborsClassifier(n_neighbors=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
KNN = RandomizedSearchCV(estimator = knn, param_distributions = param_grid_kn, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
KNN.fit(X_train, y_train)

best_KNN = KNN.best_estimator_
model14=evaluate(best_KNN,  X_test,  y_test, 'KNN')
model14.timetaken[0] = time.time() - time1
print(model14)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:   13.1s


  model  accuracy  precision  recall  f1score  rocauc  logloss  timetaken
0   KNN  0.714286   0.613636     0.5  0.55102   0.665  9.86831  14.169795
CPU times: user 1.02 s, sys: 43.5 ms, total: 1.06 s
Wall time: 14.2 s


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   13.5s finished


In [ ]:
KNN.best_params_

{'algorithm': 'auto', 'leaf_size': 200, 'p': 1}

In [ ]:
%%time
time2=time.time()
print("base model")
base_model = KNeighborsClassifier()
base_model.fit(X_train, y_train)
model4 = evaluate(base_model,  X_test,  y_test, 'KNN')
model4.timetaken[0] = time.time() - time2
print(model4)


base model
  model  accuracy  precision  ...    rocauc    logloss  timetaken
0   KNN  0.688312       0.56  ...  0.649259  10.765447   0.019674

[1 rows x 8 columns]
CPU times: user 28.3 ms, sys: 961 µs, total: 29.2 ms
Wall time: 30.4 ms


In [ ]:
from sklearn.svm import LinearSVC
lsvc=LinearSVC(random_state=42)
print(lsvc.get_params())

{'C': 1.0, 'class_weight': None, 'dual': True, 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'squared_hinge', 'max_iter': 1000, 'multi_class': 'ovr', 'penalty': 'l2', 'random_state': 42, 'tol': 0.0001, 'verbose': 0}


In [ ]:
class_weight = [int(x) for x in np.linspace(start = 10, stop = 110, num = 11)]
class_weight.append(None)
intercept_scaling = [int(x) for x in np.linspace(1, 10, num = 10)]
# Minimum number of samples required to split a node
tol = [0.00001,0.001,0.0001,0.01,0.1,1]
dual = [True, False]
# Create the random grid
param_grid_lsvc = {'class_weight':class_weight,              
               'tol': tol,
               'intercept_scaling': intercept_scaling,
               'dual': dual
               }
print(param_grid_lsvc)

{'class_weight': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'tol': [1e-05, 0.001, 0.0001, 0.01, 0.1, 1], 'intercept_scaling': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'dual': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
%%time
time1 = time.time()
lsvc =LinearSVC(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
LSVC = RandomizedSearchCV(estimator = lsvc, param_distributions = param_grid_lsvc, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
LSVC.fit(X_train, y_train)

best_LSVC= LSVC.best_estimator_
model15=evaluate(best_LSVC,  X_test,  y_test, 'LinearSVC')
model15.timetaken[0] = time.time() - time1
print(model15)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 100 candidates, totalling 300 fits
       model  accuracy  precision  ...  rocauc    logloss  timetaken
0  LinearSVC  0.707792        0.6  ...    0.66  10.092593   1.073999

[1 rows x 8 columns]
CPU times: user 418 ms, sys: 7.67 ms, total: 426 ms
Wall time: 1.09 s


[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.1s finished


In [ ]:
LSVC.best_params_

{'class_weight': None, 'dual': False, 'intercept_scaling': 8, 'tol': 1e-05}

In [ ]:
%%time
time2=time.time()
print("base model")
base_model = LinearSVC()
base_model.fit(X_train, y_train)
model5 = evaluate(base_model,  X_test,  y_test, 'LinearSVC')
model5.timetaken[0] = time.time() - time2
print(model5)


base model
       model  accuracy  precision  ...    rocauc    logloss  timetaken
0  LinearSVC  0.655844   0.571429  ...  0.522037  11.886737    0.05132

[1 rows x 8 columns]
CPU times: user 63 ms, sys: 0 ns, total: 63 ms
Wall time: 62.1 ms


In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
gnb=GaussianNB()
print(gnb.get_params())

{'priors': None, 'var_smoothing': 1e-09}


In [ ]:
priors=[X_train,y_train]
priors.append(None)
var_smoothing=[0.00001,0.001,0.0001,0.01,0.1,1]
param_grid_gnb = {'priors':priors,              
               'var_smoothing' : var_smoothing
               }
print(param_grid_gnb)

{'priors': [     Pregnancies  Glucose  BloodPressure  ...   BMI  DiabetesPedigreeFunction  Age
353            1       90             62  ...  27.2                     0.580   24
711            5      126             78  ...  29.6                     0.439   40
373            2      105             58  ...  34.9                     0.225   25
46             1      146             56  ...  29.7                     0.564   29
682            4       95             64  ...  44.6                     0.366   22
..           ...      ...            ...  ...   ...                       ...  ...
451            2      134             70  ...  28.9                     0.542   23
113            4       76             62  ...  34.0                     0.391   25
556            1       97             70  ...  38.1                     0.218   30
667           10      111             70  ...  27.5                     0.141   40
107            4      144             58  ...  29.5                     0.2

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
%%time
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
time1 = time.time()
gnb =GaussianNB()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
GNB = RandomizedSearchCV(estimator = gnb, param_distributions = param_grid_gnb, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
GNB.fit(X_train, y_train)

best_GNB= GNB.best_estimator_
model16=evaluate(best_GNB,  X_test,  y_test, 'GassuianNB')
model16.timetaken[0] = time.time() - time1
print(model16)


Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


        model  accuracy  precision  ...    rocauc    logloss  timetaken
0  GassuianNB  0.707792   0.576271  ...  0.689815  10.092629   0.493537

[1 rows x 8 columns]
CPU times: user 131 ms, sys: 1.97 ms, total: 133 ms
Wall time: 504 ms


[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:    0.5s finished


In [ ]:
GNB.best_params_

{'priors': None, 'var_smoothing': 1e-05}

In [ ]:
%%time
time2=time.time()
print("base model")
base_model = GaussianNB()
base_model.fit(X_train, y_train)
model6 = evaluate(base_model,  X_test,  y_test, 'GaussianNB')
model6.timetaken[0] = time.time() - time2
print(model6)

base model
        model  accuracy  precision  ...    rocauc    logloss  timetaken
0  GaussianNB  0.701299   0.566667  ...  0.684815  10.316912   0.017545

[1 rows x 8 columns]
CPU times: user 25.1 ms, sys: 2.07 ms, total: 27.2 ms
Wall time: 28.2 ms


In [ ]:
from sklearn.svm import SVC
svc=SVC(random_state=42)
print(svc.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': 42, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [ ]:
class_weight = [int(x) for x in np.linspace(start = 10, stop = 110, num = 11)]
class_weight.append(None)
cache_size = [int(x) for x in np.linspace(200, 300, num = 10)]
# Minimum number of samples required to split a node
tol = [0.00001,0.001,0.0001,0.01,0.1,1]
break_ties = [True, False]
# Create the random grid
param_grid_svc = {'class_weight':class_weight,              
               'tol': tol,
               'cache_size': cache_size,
               'break_ties': break_ties
               }
print(param_grid_svc)

{'class_weight': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'tol': [1e-05, 0.001, 0.0001, 0.01, 0.1, 1], 'cache_size': [200, 211, 222, 233, 244, 255, 266, 277, 288, 300], 'break_ties': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
%%time
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
time1 = time.time()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
svc=SVC(random_state=42)
SVc = RandomizedSearchCV(estimator = svc, param_distributions = param_grid_svc, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
SVc.fit(X_train, y_train)

best_SVc= SVc.best_estimator_
model17=evaluate(best_SVC,  X_test,  y_test, 'SVC')
model17.timetaken[0] = time.time() - time1
print(model17)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 100 candidates, totalling 300 fits
  model  accuracy  precision    recall   f1score    rocauc   logloss  timetaken
0   SVC  0.720779   0.648649  0.444444  0.527473  0.657222  9.644012   1.191818
CPU times: user 455 ms, sys: 9.96 ms, total: 464 ms
Wall time: 1.2 s


[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.2s finished


In [ ]:
SVc.best_params_

{'break_ties': True, 'cache_size': 211, 'class_weight': None, 'tol': 0.01}

In [ ]:
%%time
time2=time.time()
print("base model")
svc=SVC()
svc.fit(X_train, y_train)
model7 = evaluate(svc,  X_test,  y_test, 'SVC')
model7.timetaken[0] = time.time() - time2
print(model7)

base model
  model  accuracy  precision    recall   f1score    rocauc   logloss  timetaken
0   SVC  0.720779   0.648649  0.444444  0.527473  0.657222  9.644012   0.026925
CPU times: user 41.5 ms, sys: 0 ns, total: 41.5 ms
Wall time: 42.8 ms


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(random_state=42)
print(dtc.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': 'deprecated', 'random_state': 42, 'splitter': 'best'}


In [ ]:
class_weight = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
class_weight.append(None)
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
max_features.append(None)
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
# Create the random grid
param_grid_dt = {'class_weight': class_weight,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }
print(param_grid_dt)

{'class_weight': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, None], 'max_features': ['auto', 'sqrt', None], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [ ]:
%%time
time1 = time.time()
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
dt = DecisionTreeClassifier(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
DTC = RandomizedSearchCV(estimator = dt, param_distributions = param_grid_dt, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
DTC.fit(X_train, y_train)

best_DTC= DTC.best_estimator_
model18=evaluate(best_DTC,  X_test,  y_test, 'Decision Tree')
model18.timetaken[0] = time.time() - time1
print(model18)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


           model  accuracy  precision  ...    rocauc    logloss  timetaken
0  Decision Tree  0.707792   0.595745  ...  0.664259  10.092598    1.06283

[1 rows x 8 columns]
CPU times: user 449 ms, sys: 12.2 ms, total: 461 ms
Wall time: 1.07 s


[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    1.0s finished


In [ ]:
DTC.best_params_

{'class_weight': None,
 'max_depth': 60,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 10}

In [ ]:
%%time
time2=time.time()
print("base model")
dt=DecisionTreeClassifier()
dt.fit(X_train, y_train)
model9 = evaluate(dt,  X_test,  y_test, 'Decision Tree')
model9.timetaken[0] = time.time() - time2
print(model9)

base model
           model  accuracy  precision  ...    rocauc    logloss  timetaken
0  Decision Tree  0.675325   0.543478  ...  0.626481  11.213997   0.014044

[1 rows x 8 columns]
CPU times: user 24.7 ms, sys: 0 ns, total: 24.7 ms
Wall time: 25.4 ms


In [ ]:
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
best_DT =DTC.best_estimator_
df_DT = pd.concat([evaluate(best_DT,  X_test,  y_test, 'Decision treetree')], axis=0).reset_index()
df_DT = df_DT.drop('index', axis=1)
df_DT
model8=evaluate(best_DT,  X_test,  y_test, 'Decision Tree')
model8.timetaken[0] = time.time() - time1

In [ ]:
def evaluate(model, X_test, y_test):
    y_pred1 = model.predict(X_test)
    accuracy     = accuracy_score(y_test, y_pred1)
    print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.5f}%.'.format(accuracy))

    
    return accuracy
base_model = DecisionTreeClassifier( random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)
best_DTC= DTC.best_estimator_
DTC_accuracy = evaluate(best_DTC, X_test, y_test)
print('Improvement of {:0.5f}%.'.format( 100 * (base_accuracy - DTC_accuracy) / base_accuracy))


Model Performance
Accuracy = 0.66883%.
Model Performance
Accuracy = 0.70779%.
Improvement of -5.82524%.


In [ ]:
df_randomized= pd.concat([model11,model12,model13,model14,model15,model16,model17,model18],axis = 0).reset_index()
df_randomized.drop('index', axis=1, inplace=True)
df_randomized

,model,accuracy,precision,recall,f1score,rocauc,logloss,timetaken
0,RandomForest,0.759740,0.680851,0.592593,0.633663,0.721296,8.298355,441.047501
1,Extra Tree,0.727273,0.620000,0.574074,0.596154,0.692037,9.419765,377.264013
2,Logistic Regression,0.701299,0.583333,0.518519,0.549020,0.659259,10.316881,7.526623
3,KNN,0.714286,0.613636,0.500000,0.551020,0.665000,9.868310,14.169795
4,LinearSVC,0.707792,0.600000,0.500000,0.545455,0.660000,10.092593,1.073999
5,GassuianNB,0.707792,0.576271,0.629630,0.601770,0.689815,10.092629,0.493537
6,SVC,0.720779,0.648649,0.444444,0.527473,0.657222,9.644012,1.191818
7,Decision Tree,0.707792,0.595745,0.518519,0.554455,0.664259,10.092598,1.062830


In [ ]:
df_base= pd.concat([model1,model2,model3,model4,model5,model6,model7,model9],axis = 0).reset_index()
df_base.drop('index', axis=1, inplace=True)
df_base

,model,accuracy,precision,recall,f1score,rocauc,logloss,timetaken
0,RandomForest,0.740260,0.652174,0.555556,0.600000,0.697778,8.971194,0.223264
1,Extra Tree,0.720779,0.612245,0.555556,0.582524,0.682778,9.644043,0.176967
2,Logistic Regression,0.740260,0.659091,0.537037,0.591837,0.693519,8.971189,0.187247
3,KNN,0.688312,0.560000,0.518519,0.538462,0.649259,10.765447,0.019674
4,LinearSVC,0.655844,0.571429,0.074074,0.131148,0.522037,11.886737,0.051320
5,GaussianNB,0.701299,0.566667,0.629630,0.596491,0.684815,10.316912,0.017545
6,SVC,0.720779,0.648649,0.444444,0.527473,0.657222,9.644012,0.026925
7,Decision Tree,0.675325,0.543478,0.462963,0.500000,0.626481,11.213997,0.014044


In [ ]:
df_baseline1

,model,accuracy,precision,recall,f1score,rocauc,logloss
0,Logistic Regresion,0.740260,0.659091,0.537037,0.591837,0.693519,8.971189
1,GaussianNB,0.701299,0.566667,0.629630,0.596491,0.684815,10.316912
2,KNN,0.688312,0.560000,0.518519,0.538462,0.649259,10.765447
3,DecisionTree,0.675325,0.543478,0.462963,0.500000,0.626481,11.213997
4,ExtraTrees,0.714286,0.600000,0.555556,0.576923,0.677778,9.868326
5,RandomForest,0.753247,0.666667,0.592593,0.627451,0.716296,8.522638
6,LinearSVC,0.649351,0.500000,0.074074,0.129032,0.517037,12.111020
7,SVC,0.720779,0.648649,0.444444,0.527473,0.657222,9.644012


In [ ]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import sys
sys.path.append('../..')

from sklearn_nature_inspired_algorithms.model_selection.nature_inspired_search_cv import NatureInspiredSearchCV
from sklearn_nature_inspired_algorithms.helpers import score_by_generation_lineplot

BAT ALGORITHM

In [ ]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 800}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
time1 = time.time()
def evaluate1(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_modell = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_modell

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier
time1 = time.time()
param_grid={'bootstrap': [True],
 'max_depth': [10,20,30,100],
 'max_features': [2,3],
 'min_samples_leaf': [1,2,3],
 'n_estimators': [800,900,1000,1200,1500],
 'min_samples_split': [3,5,8]
 
    
}
clf_1 = RandomForestClassifier(random_state=42)
nia_search = NatureInspiredSearchCV(
    clf_1,
    param_grid=param_grid,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen= 20,
    max_stagnating_gen=30,
    runs=3,
    random_state=42,
    
    # or any number if you want same results on each run
)


nia_search.fit(X_train, y_train)
print("Bat model")
ba_random= nia_search.best_estimator_
model21=evaluate1(ba_random,  X_test,  y_test, 'Random Forest')
model21.timetaken[0] = time.time() - time1
print(model21)

Fitting at most 360 candidates
Bat model
           model  accuracy  precision  ...    rocauc   logloss  timetaken
0  Random Forest  0.753247   0.673913  ...  0.712037  8.522633  2563.5305

[1 rows x 8 columns]
CPU times: user 42min 31s, sys: 13.6 s, total: 42min 44s
Wall time: 42min 43s


In [ ]:
nia_search.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 800}

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
time1 = time.time()
def evaluate1(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_modell = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_modell
print("base model")
base_model = RandomForestClassifier()
base_model.fit(X_train, y_train)
base_accuracy = evaluate1(base_model, X_test, y_test,'random')
print(base_accuracy)


base model
    model  accuracy  precision  ...    rocauc  logloss     timetaken
0  random  0.727273      0.625  ...  0.687778  9.41976  1.636629e+09

[1 rows x 8 columns]
CPU times: user 245 ms, sys: 0 ns, total: 245 ms
Wall time: 250 ms


In [ ]:
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
ba_random = nia_search.best_estimator_
df_ba = pd.concat([evaluate(ba_random,  X_test,  y_test, 'RandomForest')], axis=0).reset_index()
df_ba = df_ba.drop('index', axis=1)
print(df_ba)
model11=evaluate(ba_random,  X_test,  y_test, 'RandomForest')
model11.timetaken[0] = time.time() - time1



          model  accuracy  precision  ...    rocauc   logloss     timetaken
0  RandomForest  0.753247   0.673913  ...  0.712037  8.522633  1.636629e+09

[1 rows x 8 columns]


In [ ]:
def evaluate(model, X_test, y_test):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)
  print('Model Performance')
  print('precision={:0.3f}%.'.format(precision))
  print('Accuracy = {:0.3f}%.'.format(accuracy))

    
  return accuracy
  return precision
base_model = RandomForestClassifier()
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)
best_random = nia_search.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)
print('Improvement of {:0.3f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Model Performance
precision=0.612%.
Accuracy = 0.721%.
Model Performance
precision=0.674%.
Accuracy = 0.753%.
Improvement of 4.505%.


In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier
time1 = time.time()
param_grid={'bootstrap': [True],
 'max_depth': [None],
 'max_features': [2,3],
 'min_samples_leaf': [1,2,3],
 'n_estimators': [800,900,1000,1200,1500],
 'min_samples_split': [3,5,8]
 
    
}
clf_1 = RandomForestClassifier(random_state=42)
nia_search = NatureInspiredSearchCV(
    clf_1,
    param_grid=param_grid,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen= 20,
    max_stagnating_gen=30,
    runs=3,
    random_state=42,
    
    # or any number if you want same results on each run
)

nia_search.fit(X_train, y_train)



Fitting at most 90 candidates
CPU times: user 15min 37s, sys: 4.93 s, total: 15min 42s
Wall time: 15min 41s


In [ ]:
nia_search.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 3,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 800}

In [ ]:
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
ba_random = nia_search.best_estimator_
df_ba = pd.concat([evaluate(ba_random,  X_test,  y_test, 'RandomForest')], axis=0).reset_index()
df_ba = df_ba.drop('index', axis=1)
print(df_ba)
model11=evaluate(ba_random,  X_test,  y_test, 'RandomForest')
model11.timetaken[0] = time.time() - time1



          model  accuracy  precision  ...    rocauc   logloss     timetaken
0  RandomForest  0.733766   0.644444  ...  0.688519  9.195472  1.636623e+09

[1 rows x 8 columns]


In [ ]:
def evaluate(model, X_test, y_test):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)
  print('Model Performance')
  print('precision={:0.3f}%.'.format(precision))
  print('Accuracy = {:0.3f}%.'.format(accuracy))

    
  return accuracy
  return precision
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)
best_random = nia_search.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)
print('Improvement of {:0.3f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Model Performance
precision=0.585%.
Accuracy = 0.695%.
Model Performance
precision=0.644%.
Accuracy = 0.734%.
Improvement of 5.607%.


In [ ]:
LR.best_params_

{'C': 6.0,
 'class_weight': 7,
 'dual': False,
 'intercept_scaling': 1,
 'max_iter': 490,
 'n_jobs': 7,
 'solver': 'newton-cg',
 'tol': 1e-05,
 'verbose': 2}

In [ ]:
%%time
time1 = time.time()
param_grid={'class_weight':[5,8,9,10,30],
 'dual': [False],
 'intercept_scaling':[1,2,3,4,5,6],
 'max_iter': [1200,1000,1300,1500,2000],
 'solver':['newton-cg'],
  'verbose': [ 2,1,3,4]
 }

clf_2 = LogisticRegression(random_state=42)

nia_search_lr = NatureInspiredSearchCV(
    clf_2,
    param_grid=param_grid,
    algorithm='ba',  # bat algorithm
    population_size=100,
    max_n_gen=150,
    max_stagnating_gen=100,
    runs=3,
    random_state=42, # or any number if you want same results on each run
)

nia_search_lr.fit(X_train, y_train)



Fitting at most 600 candidates


Streaming output truncated to the last 5000 lines.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend

CPU times: user 1min 21s, sys: 1.32 s, total: 1min 22s
Wall time: 1min 22s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [ ]:
nia_search_lr.best_params_

{'class_weight': 8,
 'dual': False,
 'intercept_scaling': 5,
 'max_iter': 1300,
 'solver': 'newton-cg',
 'verbose': 2}

In [ ]:
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
ba_lr = nia_search_lr.best_estimator_
df_ba = pd.concat([evaluate(ba_lr,  X_test,  y_test, 'RandomForest')], axis=0).reset_index()
df_ba = df_ba.drop('index', axis=1)
print(df_ba)
model12=evaluate(ba_lr,  X_test,  y_test, 'Logistic Regression')
model12.timetaken[0] = time.time() - time1



          model  accuracy  precision  ...    rocauc    logloss     timetaken
0  RandomForest  0.701299   0.583333  ...  0.659259  10.316881  1.636635e+09

[1 rows x 8 columns]


In [ ]:
def evaluate(model, X_test, y_test):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)
  print('Model Performance')
  print('precision={:0.3f}%.'.format(precision))
  print('Accuracy = {:0.3f}%.'.format(accuracy))

    
  return accuracy
  return precision
base_model = LogisticRegression( random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)
best_lr = nia_search_lr.best_estimator_
lr_accuracy = evaluate(best_lr, X_test, y_test)
print('Improvement of {:0.3f}%.'.format( 100 * (lr_accuracy - base_accuracy) / base_accuracy))

Model Performance
precision=0.659%.
Accuracy = 0.740%.
Model Performance
precision=0.583%.
Accuracy = 0.701%.
Improvement of -5.263%.


In [ ]:
ETC.best_params_

{'bootstrap': False,
 'max_depth': 70,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 1600}

In [ ]:
%%time
import sys
sys.path.append('../..')
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import ExtraTreesClassifier
time1 = time.time()
param_grid={'bootstrap': [True],
 'max_depth': [80,100,110,120],
 'max_features': ['auto','sqrt'],
 'min_samples_leaf': [1,2,3,5],
 'n_estimators': [1800,2000,2100,2200],
 'min_samples_split': [3,5,8,10]
 
    
}
clf_3 = ExtraTreesClassifier(random_state=42)

nia_search_etc = NatureInspiredSearchCV(
    clf_3,
    param_grid=param_grid,
    algorithm='ba', #  bat algorithm
    population_size=15,
    max_n_gen=5,
    max_stagnating_gen=5,
    runs=3,
    random_state=42, # or any number if you want same results on each run
)

nia_search_etc.fit(X_train, y_train)


Fitting at most 512 candidates
CPU times: user 23min 9s, sys: 8.53 s, total: 23min 17s
Wall time: 23min 16s


In [ ]:
nia_search_etc.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 2000}

In [ ]:
def evaluate(model, X_test , y_test,modelname):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)

  df_model1 = pd.DataFrame({  'model'        : [modelname],
                             'accuracy'     : [accuracy],
                             'precision'    : [precision],
                             'recall'       : [recall],
                             'f1score'      : [f1score],
                             'rocauc'       : [rocauc],
                             'logloss'      : [logloss],
                             'timetaken'    : [time1]   
                                }) 
  return df_model1
ba_etc = nia_search_etc.best_estimator_
df_ba = pd.concat([evaluate(ba_etc,  X_test,  y_test, 'RandomForest')], axis=0).reset_index()
df_ba = df_ba.drop('index', axis=1)
print(df_ba)
model12=evaluate(ba_etc,  X_test,  y_test, 'Extra Tree')
model12.timetaken[0] = time.time() - time1



          model  accuracy  precision  ...    rocauc   logloss     timetaken
0  RandomForest   0.75974   0.680851  ...  0.721296  8.298355  1.636635e+09

[1 rows x 8 columns]


In [ ]:
def evaluate(model, X_test, y_test):
  y_pred1=model.predict(X_test)
  precision    = precision_score(y_test, y_pred1)
  recall       = recall_score(y_test, y_pred1)
  f1score      = f1_score(y_test, y_pred1) 
  rocauc       = roc_auc_score(y_test, y_pred1)
  logloss      = log_loss(y_test, y_pred1)
  accuracy     = accuracy_score(y_test, y_pred1)
  print('Model Performance')
  print('precision={:0.3f}%.'.format(precision))
  print('Accuracy = {:0.3f}%.'.format(accuracy))

    
  return accuracy
  return precision
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)
best_etc = nia_search_etc.best_estimator_
etc_accuracy = evaluate(best_etc, X_test, y_test)
print('Improvement of {:0.3f}%.'.format( 100 * (etc_accuracy - base_accuracy) / base_accuracy))

In [ ]:
y_pred = nia_search_etc.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

model2 = modell_report(nia_search_etc, X_train, X_test, y_train, y_test,y_pred, 'Extra tree')
model2.timetaken[0] = time.time() - time1
model2

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import ExtraTreesClassifier
time1 = time.time()
clf_3 = ExtraTreesClassifier(random_state=42)

nia_search_etc = NatureInspiredSearchCV(
    clf_3,
    param_grid_et,
    algorithm='ba', #  bat algorithm
    population_size=25,
    max_n_gen=10,
    max_stagnating_gen=15,
    runs=3,
    random_state=42, # or any number if you want same results on each run
)

nia_search_etc.fit(X_train, y_train)


In [ ]:
nia_search_etc.best_params_

In [ ]:
y_pred = nia_search_etc.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

model2 = modell_report(nia_search_etc, X_train, X_test, y_train, y_test,y_pred, 'Extra tree')
model2.timetaken[0] = time.time() - time1
model2

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
kn=KNeighborsClassifier(n_neighbors=None)
print(kn.get_params())

In [ ]:
leaf_size = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)]
# Number of features to consider at every split
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
# Maximum number of levels in tree
p = [int(x) for x in np.linspace(1, 20, num = 20)]
param_grid_kn = {'leaf_size': leaf_size,
               'p': p,
               'algorithm': algorithm,
               }
print(param_grid_kn)

NameError: ignored

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.neighbors import KNeighborsClassifier
time1 = time.time()
clf_4 = KNeighborsClassifier(n_neighbors=5)

nia_search_kn = NatureInspiredSearchCV(
    clf_4,
    param_grid_kn,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=10,
    runs=3,
    random_state=5, # or any number if you want same results on each run
)

nia_search_kn.fit(X_train, y_train)

In [ ]:
nia_search_kn.best_params_

In [ ]:
y_pred = nia_search_kn.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

model3 = modell_report(nia_search_kn, X_train, X_test, y_train, y_test,y_pred, 'Knn')
model3.timetaken[0] = time.time() - time1
model3

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.neighbors import KNeighborsClassifier
time1 = time.time()
clf_4 = KNeighborsClassifier(n_neighbors=5)

nia_search_kn = NatureInspiredSearchCV(
    clf_4,
    param_grid_kn,
    algorithm='ba', #  bat algorithm
    population_size=100,
    max_n_gen=150,
    max_stagnating_gen=20,
    runs=3,
    random_state=5, # or any number if you want same results on each run
)

nia_search_kn.fit(X_train, y_train)

In [ ]:
nia_search_kn.best_params_

In [ ]:
y_pred = nia_search_kn.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

model3 = modell_report(nia_search_kn, X_train, X_test, y_train, y_test,y_pred, 'Knn')
model3.timetaken[0] = time.time() - time1
model3

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.neighbors import KNeighborsClassifier
time1 = time.time()
clf_4 = KNeighborsClassifier(n_neighbors=5)

nia_search_kn_hba = NatureInspiredSearchCV(
    clf_4,
    param_grid_kn,
    algorithm='hba', #  bat algorithm
    population_size=100,
    max_n_gen=150,
    max_stagnating_gen=20,
    runs=3,
    random_state=5, # or any number if you want same results on each run
)

nia_search_kn_hba.fit(X_train, y_train)

In [ ]:
nia_search_kn_hba.best_params_

In [ ]:
nia_search_kn_hba.best_estimator_

In [ ]:
print("HBA")
y_pred = nia_search_kn_hba.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

model3_hba = modell_report(nia_search_kn_hba, X_train, X_test, y_train, y_test,y_pred, 'Knn')
model3_hba.timetaken[0] = time.time() - time1
model3_hba

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.neighbors import KNeighborsClassifier
time1 = time.time()
clf_4 = KNeighborsClassifier(n_neighbors=5)

nia_search_kn_hsaba = NatureInspiredSearchCV(
    clf_4,
    param_grid_kn,
    algorithm='hsaba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=10,
    runs=3,
    random_state=5, # or any number if you want same results on each run
)

nia_search_kn_hsaba.fit(X_train, y_train)

In [ ]:
nia_search_kn_hsaba.best_params_

In [ ]:
print("HSABA")
y_pred = nia_search_kn_hsaba.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

model3_hsaba = modell_report(nia_search_kn_hsaba, X_train, X_test, y_train, y_test,y_pred, 'Knn')
model3_hsaba.timetaken[0] = time.time() - time1
model3_hsaba

In [ ]:
pip install niapy==2.0.0rc17

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)

In [ ]:
print(param_grid)

In [ ]:
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier
from NiaPy.algorithms.basic import GeneticAlgorithm
clf = RandomForestClassifier(random_state=42)

algorithm = GeneticAlgorithm() # when custom algorithm is provided random_state is ignored
#algorithm.set_parameters(NP=50, Ts=5, Mr=0.25)

nia_search_gen = NatureInspiredSearchCV(
    clf,
    param_grid,
    algorithm=algorithm,
    population_size=5,
    max_n_gen=3,
    max_stagnating_gen=3,
    runs=3,
)

nia_search_gen.fit(X_train, y_train)

In [ ]:
nia_search_gen.best_params_

In [ ]:
print("GA")
y_pred = nia_search_gen.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

model3_ga = modell_report(nia_search_gen, X_train, X_test, y_train, y_test,y_pred, 'Knn')
model3_ga.timetaken[0] = time.time() - time1
model3_ga

In [ ]:
from sklearn.svm import LinearSVC
lsvc=LinearSVC(random_state=42)
print(lsvc.get_params())

In [ ]:
class_weight = [int(x) for x in np.linspace(start = 10, stop = 110, num = 11)]
class_weight.append(None)
intercept_scaling = [int(x) for x in np.linspace(1, 10, num = 10)]
# Minimum number of samples required to split a node
tol = [0.00001,0.001,0.0001,0.01,0.1,1]
dual = [True, False]
# Create the random grid
param_grid_lsvc = {'class_weight':class_weight,              
               'tol': tol,
               'intercept_scaling': intercept_scaling,
               'dual': dual
               }
print(param_grid_lsvc)

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
time1 = time.time()
clf_6 = LinearSVC(random_state=42)

nia_search_lsvc = NatureInspiredSearchCV(
    clf_6,
    param_grid_lsvc,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=10,
    runs=3,
    random_state=None, # or any number if you want same results on each run
)

nia_search_lsvc.fit(X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
print(dt.get_params())

In [ ]:
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

class_weight = [int(x) for x in np.linspace(start = 10, stop = 110, num = 11)]
class_weight.append(None)
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
#bootstrap = [True, False]
# Create the random grid
param_grid_dtc = {'class_weight':class_weight,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               #'bootstrap': bootstrap
               }
print(param_grid_dtc)

In [ ]:
time1 = time.time()
clf_5 = DecisionTreeClassifier(random_state=42)
nia_search_dtc = NatureInspiredSearchCV(
    clf_5,
    param_grid_dtc,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=10,
    runs=3,
    random_state=42, # or any number if you want same results on each run
)

nia_search_dtc.fit(X_train, y_train)

In [ ]:
nia_search_dt.best_params_

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(random_state=42)
print(lr.get_params())

In [ ]:
max_iter = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
solver=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
class_weight = [int(x) for x in np.linspace(10, 110, num = 11)]
class_weight.append(None)
intercept_scaling = [1, 2, 4]
dual = [True, False]
multi_class=['auto', 'ovr', 'multinomial']
#tol= [0.00001,0.0001,0.001,0.01,0.1,0,1]
# Create the random grid
param_grid = {'max_iter': max_iter,
               'class_weight': class_weight,
               'solver': solver,
               'intercept_scaling': intercept_scaling,
               'dual': dual,
               #'multi_class' : multi_class
              }
print(param_grid)

In [ ]:
pip install niapy==2.0.0rc17

In [ ]:
%%time
time1 = time.time()

clf_2 = LogisticRegression(random_state=42)

nia_search = NatureInspiredSearchCV(
    clf_2,
    param_grid,
    algorithm='ba', # bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=10,
    runs=3,
    random_state=42, # or any number if you want same results on each run
)

nia_search.fit(X_train, y_train)



In [ ]:
nia_search.best_params_

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(random_state=42)
print(et.get_params())

In [ ]:
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import RandomForestClassifier
time1 = time.time()
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid_et = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid_et)

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.ensemble import ExtraTreesClassifier
time1 = time.time()
clf_3 = ExtraTreesClassifier(random_state=42)

nia_search_etc = NatureInspiredSearchCV(
    clf_3,
    param_grid_et,
    algorithm='ba', #  bat algorithm
    population_size=15,
    max_n_gen=5,
    max_stagnating_gen=5,
    runs=3,
    random_state=42, # or any number if you want same results on each run
)

nia_search_etc.fit(X_train, y_train)


In [ ]:
nia_search_etc.best_params_

In [ ]:
y_pred = nia_search_etc.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

model1 = modell_report(nia_search_etc, X_train, X_test, y_train, y_test,y_pred, 'Extra tree')
model1.timetaken[0] = time.time() - time1
model1

In [ ]:
def evaluate(model, X_test, y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
best_etc = nia_search_etc.best_estimator_
etc_accuracy = evaluate(best_etc, X_test,y_test)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
kn=KNeighborsClassifier(n_neighbors=None)
print(kn.get_params())

In [ ]:
leaf_size = [int(x) for x in np.linspace(start = 10, stop = 200, num = 20)]
# Number of features to consider at every split
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
# Maximum number of levels in tree
p = [int(x) for x in np.linspace(1, 20, num = 20)]
param_grid_kn = {'leaf_size': leaf_size,
               'p': p,
               'algorithm': algorithm,
               }
print(param_grid_kn)

In [ ]:
%%time
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.neighbors import KNeighborsClassifier
time1 = time.time()
clf_4 = KNeighborsClassifier(n_neighbors=5)

nia_search = NatureInspiredSearchCV(
    clf_4,
    param_grid_kn,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=10,
    runs=3,
    random_state=5, # or any number if you want same results on each run
)

nia_search.fit(X_train, y_train)

In [ ]:
nia_search.best_params_

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
print(dt.get_params())

In [ ]:
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

class_weight = [int(x) for x in np.linspace(start = 10, stop = 110, num = 11)]
class_weight.append(None)
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
#bootstrap = [True, False]
# Create the random grid
param_grid_dt = {'class_weight':class_weight,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               #'bootstrap': bootstrap
               }
print(param_grid_dt)

In [ ]:

from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV
from sklearn.tree import DecisionTreeClassifier
time1 = time.time()
clf_5 = DecisionTreeClassifier(random_state=42)

nia_search = NatureInspiredSearchCV(
    clf_5,
    param_grid_dt,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=10,
    runs=3,
    random_state=42, # or any number if you want same results on each run
)

nia_search.fit(X_train, y_train)

In [ ]:
nia_search.best_params_

In [ ]:
from sklearn.svm import LinearSVC
lsvc=LinearSVC(random_state=42)
print(lsvc.get_params())

In [ ]:
from sklearn_nature_inspired_algorithms.model_selection import NatureInspiredSearchCV

class_weight = [int(x) for x in np.linspace(start = 10, stop = 110, num = 11)]
class_weight.append(None)
intercept_scaling = [int(x) for x in np.linspace(1, 10, num = 10)]
# Minimum number of samples required to split a node
tol = [0.00001,0.001,0.0001,0.01,0.1,1]
dual = [True, False]
# Create the random grid
param_grid_lsvc = {'class_weight':class_weight,              
               'tol': tol,
               'intercept_scaling': intercept_scaling,
               'dual': dual
               }
print(param_grid_lsvc)

In [ ]:
time1 = time.time()
clf_6 = LinearSVC(random_state=42)

nia_search = NatureInspiredSearchCV(
    clf_6,
    param_grid_lsvc,
    algorithm='ba', #  bat algorithm
    population_size=50,
    max_n_gen=100,
    max_stagnating_gen=10,
    runs=3,
    random_state=None, # or any number if you want same results on each run
)

nia_search.fit(X_train, y_train)